In [1]:
from langchain_community.document_loaders import TextLoader
loader=TextLoader("simple.txt")
text_document=loader.load()
text_document

[Document(metadata={'source': 'simple.txt'}, page_content='Excellent question ðŸš€ This is the **heart of RAG (Retrieval Augmented Generation)**.\nLetâ€™s break it down step by step so itâ€™s crystal clear:\n\n---\n\n## ðŸ”¹ 1. What is a Retriever in RAG?\n\nIn **RAG (Retrieval-Augmented Generation)**, the LLM doesnâ€™t just generate text blindly â€” it **retrieves relevant documents** from a knowledge base first, then uses them to answer.\n\nThe component that **fetches relevant documents** from a vector store (like FAISS, Pinecone, Weaviate, Chroma) is called a **Retriever**.\n\nðŸ‘‰ In LangChain, **retrievers** are a **standardized interface** around search methods (similarity search, Max Marginal Relevance, hybrid search, etc.).\n\n---\n\n## ðŸ”¹ 2. Similarity Search vs Retriever\n\n### **(a) Similarity Search**\n\n* A low-level method.\n* You directly ask your **vector database**:\n\n  ```python\n  results = vectorstore.similarity_search(query, k=3)\n  ```\n* It finds the **k most

In [2]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
loader= WebBaseLoader(web_path="http://lilianweng.github.io/posts/2024-11-28-reward-hacking/",
                    bs_kwargs=dict( parse_only=bs4.SoupStrainer(
                    class_=("post-title","post_content","post_header")
))) 
text_documents=loader.load()
text_documents

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'http://lilianweng.github.io/posts/2024-11-28-reward-hacking/'}, page_content='\n      Reward Hacking in Reinforcement Learning\n    ')]

In [3]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("LLM.pdf")
docs=loader.load()
docs

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-08-24T07:43:22+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-08-24T07:43:22+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'LLM.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Large Language Models (LLMs) and\nRetrieval-Augmented Generation (RAG)\nLarge Language Models (LLMs) are a class of artificial intelligence models designed to understand\nand generate human-like language. They are trained on massive datasets consisting of text from\nbooks, websites, articles, and more. LLMs use deep learning architectures, primarily transformers,\nto capture semantic meaning and context across long sequences of text. Key characteristics of\nLLMs include: - **Token-based Processing**: Text is broken down into tokens (sub-words or\ncharacters) that the model processes. - **Context Windows**

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
documents=text_splitter.split_documents(docs)
documents[:5]

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-08-24T07:43:22+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-08-24T07:43:22+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'LLM.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Large Language Models (LLMs) and\nRetrieval-Augmented Generation (RAG)\nLarge Language Models (LLMs) are a class of artificial intelligence models designed to understand\nand generate human-like language. They are trained on massive datasets consisting of text from\nbooks, websites, articles, and more. LLMs use deep learning architectures, primarily transformers,\nto capture semantic meaning and context across long sequences of text. Key characteristics of'),
 Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-08-24T07:43:22+00:00'

In [5]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
db=Chroma.from_documents(documents[:3],OllamaEmbeddings())//instad of chroma we can use FAISS also 


C:\Users\abhij\AppData\Local\Temp\ipykernel_18820\2798681641.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  db=Chroma.from_documents(documents[:3],OllamaEmbeddings())


In [7]:
query="What is RAG ?"
result=db.similarity_search(query)
# result 
result[0].page_content

'Fine-tuning**: LLMs are pretrained on large datasets and later fine-tuned for specific tasks like\nsummarization, coding, or question answering. - **Applications**: Chatbots, machine translation,\ntext summarization, coding assistants, and more. However, LLMs have limitations: - They may\n"hallucinate" or generate plausible but incorrect information. - Knowledge is restricted to their\ntraining data cutoff date. - Large computational and memory requirements.'

In [8]:
from langchain_community.llms import Ollama
llm=Ollama(model="llama2")
llm

C:\Users\abhij\AppData\Local\Temp\ipykernel_18820\2415645411.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(model="llama2")


Ollama()

In [9]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
I will tip you $1000 if the user finds the answer helpful.

<context>
{context}
</context>

Question: {input}
""")


In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
documents_chain=create_stuff_documents_chain(llm,prompt) 

In [13]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000147F83D8690>, search_kwargs={})

In [14]:
"""
Retrieval chain: This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,documents_chain)


In [ ]:
response=retrieval_chain.invoke({"input":"What is LLM"})

In [ ]:
response["answer"]

C:\Users\abhij\AppData\Local\Temp\ipykernel_12884\3833718216.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama2:chat")
